In [ ]:
%reset -f

### Pildora de Analítica Espacial ***-Partimos del trabajo de un dataset anterior, que no podemos hacer público***


Análisis de Densidad Espacial de Hoteles (Mapa de calor por Barrio)
¿Qué hace?
Calcula cuántos hoteles hay por barrio  y crea un mapa de calor (choropleth) que muestra la densidad turística.

Ayuda a definir más si algunas zonas tienen muchos hoteles pero solo una oficina, lo que podría indicar sobrecarga de servicio o necesidad de refuerzo, asi identificamos las áreas más de turismo y con menos actividad laboral

In [ ]:
# Cargar Librerías

!apt-get update -qq
!apt-get install -y -qq libspatialindex-dev libproj-dev libgeos-dev libgdal-dev
!pip install -q geopandas folium scikit-learn matplotlib

# Importar librerías
import pandas as pd
import geopandas as gpd
import numpy as np
import matplotlib.pyplot as plt
from folium import LayerControl
import folium
from sklearn.cluster import KMeans

print("✅ Librerías cargadas correctamente")

W: Skipping acquire of configured file 'main/source/Sources' as repository 'https://r2u.stat.illinois.edu/ubuntu jammy InRelease' does not seem to provide it (sources.list entry misspelt?)
Selecting previously unselected package libspatialindex6:amd64.
(Reading database ... 126371 files and directories currently installed.)
Preparing to unpack .../libspatialindex6_1.9.3-2_amd64.deb ...
Unpacking libspatialindex6:amd64 (1.9.3-2) ...
Selecting previously unselected package libspatialindex-c6:amd64.
Preparing to unpack .../libspatialindex-c6_1.9.3-2_amd64.deb ...
Unpacking libspatialindex-c6:amd64 (1.9.3-2) ...
Selecting previously unselected package libspatialindex-dev:amd64.
Preparing to unpack .../libspatialindex-dev_1.9.3-2_amd64.deb ...
Unpacking libspatialindex-dev:amd64 (1.9.3-2) ...
Setting up libspatialindex6:amd64 (1.9.3-2) ...
Setting up libspatialindex-c6:amd64 (1.9.3-2) ...
Setting up libspatialindex-dev:amd64 (1.9.3-2) ...
Processing triggers for libc-bin (2.35-0ubuntu3.8) .

In [16]:
# Conectar con Google Drive
from google.colab import drive
drive.mount('/content/drive')

# Ajusta las rutas a tus archivos (Ponga su ruta adecuada)
ruta_hoteles1 = '/content/drive/MyDrive/Colab Notebooks/01-AE/data/raw/hoteles1.shp'
ruta_hoteles2 = '/content/drive/MyDrive/Colab Notebooks/01-AE/data/raw/hoteles2.shp'
ruta_hoteles3 = '/content/drive/MyDrive/Colab Notebooks/01-AE/data/raw/hoteles3.shp'
ruta_hoteles4 = '/content/drive/MyDrive/Colab Notebooks/01-AE/data/raw/hoteles4.shp'
ruta_hoteles5 = '/content/drive/MyDrive/Colab Notebooks/01-AE/data/raw/hoteles5.shp'
ruta_oficinas = '/content/drive/MyDrive/Colab Notebooks/01-AE/data/raw/oficinaturismo.shp'
ruta_barrios = '/content/drive/MyDrive/Colab Notebooks/01-AE/data/raw/200001543.shp'

# Cargar de los datos
i_data_hoteles1 = gpd.read_file(ruta_hoteles1, encoding="latin1")
i_data_hoteles2 = gpd.read_file(ruta_hoteles2, encoding="latin1")
i_data_hoteles3 = gpd.read_file(ruta_hoteles3, encoding="latin1")
i_data_hoteles4 = gpd.read_file(ruta_hoteles4, encoding="latin1")
i_data_hoteles5 = gpd.read_file(ruta_hoteles5, encoding="latin1")
i_data_oficinaturismo = gpd.read_file(ruta_oficinas, encoding="latin1")
i_data_barrios = gpd.read_file(ruta_barrios, encoding="latin1")



Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [20]:
# Unir todos los archivos de hoteles en uno solo
i_data_hoteles = pd.concat([
    i_data_hoteles1,
    i_data_hoteles2,
    i_data_hoteles3,
    i_data_hoteles4,
    i_data_hoteles5
], ignore_index=True)

print(f"✅ Total de hoteles cargados: {len(i_data_hoteles)}")

✅ Total de hoteles cargados: 474


In [22]:
# Ver las columnas del archivo de los barrios
print("📋 Columnas disponibles en i_data_barrios:")
print(i_data_barrios.columns.tolist())

📋 Columnas disponibles en i_data_barrios:
['CODBDT', 'GEOCODIGO', 'DESBDT', 'geometry']


In [23]:
# Filtrar barrios del distrito Centro Madrid
i_data_centro = i_data_barrios[
    (i_data_barrios['GEOCODIGO'] >= "079011") &
    (i_data_barrios['GEOCODIGO'] <= "079076")
].copy()

# Comprobar resultados
print(f"✅ Barrios del distrito Centro: {len(i_data_centro)}")

# Mostrar columnas
print("\n📋 Nombres de barrios en el Centro:")
i_data_centro[['GEOCODIGO', 'DESBDT']].reset_index(drop=True)

✅ Barrios del distrito Centro: 43

📋 Nombres de barrios en el Centro:


,GEOCODIGO,DESBDT
0,079011,011 Palacio
1,079012,012 Embajadores
2,079013,013 Cortes
3,079014,014 Justicia
4,079015,015 Universidad
5,079016,016 Sol
6,079021,021 Imperial
7,079022,022 Las Acacias
8,079023,023 La Chopera
9,079024,024 Legazpi


In [25]:
# Filtrar hoteles dentro del Centro
i_data_hoteles_centro = gpd.sjoin(
    i_data_hoteles,
    i_data_centro[['GEOCODIGO', 'geometry']],
    how="inner",
    predicate="within"
)

# Filtrar oficinas de turismo del Centro
i_data_oficinaturismo_centro = gpd.sjoin(
    i_data_oficinaturismo,
    i_data_centro[['GEOCODIGO', 'geometry']],
    how="inner",
    predicate="within"
)

print(f"✅ Hoteles en el Centro: {len(i_data_hoteles_centro)}")
print(f"✅ Oficinas de turismo en el Centro: {len(i_data_oficinaturismo_centro)}")

✅ Hoteles en el Centro: 214
✅ Oficinas de turismo en el Centro: 6


In [36]:
# Recuento de los hoteles por barrio

# Comprobar de que i_data_hoteles_centro no tenga columnas extrañas
# Eliminamos columnas raras que podrían venir del join anterior
if 'index_right' in i_data_hoteles_centro.columns:
    i_data_hoteles_centro = i_data_hoteles_centro.drop(columns=['index_right'])

# También aseguramos que solo tenga las columnas necesarias
i_data_hoteles_centro = i_data_hoteles_centro[['ETIQUETA', 'geometry']].copy()

# Solo las columnas necesarias de i_data_centro
barrios_centro_simple = i_data_centro[['GEOCODIGO', 'DESBDT', 'geometry']].copy()

# Realizar el join limpio
hoteles_por_barrio = gpd.sjoin(
    i_data_hoteles_centro,
    barrios_centro_simple,
    how="inner",
    predicate="within"
)

# Recuento de hoteles por barrio
conteo_hoteles = hoteles_por_barrio.groupby('GEOCODIGO').size().reset_index(name='num_hoteles')

print("✅ Recuento de hoteles por barrio completado sin errores.")
conteo_hoteles.head()

✅ Recuento de hoteles por barrio completado sin errores.


,GEOCODIGO,num_hoteles
0,079011,20
1,079012,8
2,079013,31
3,079014,11
4,079015,15


In [37]:
# Eliminar columnas problemáticas
if 'index_right' in i_data_hoteles_centro.columns:
    i_data_hoteles_centro.drop(columns=['index_right'], inplace=True)

# Seleccionar solo columnas esenciales para evitar duplicados
hoteles_clean = i_data_hoteles_centro[['geometry']].copy()
barrios_clean = i_data_centro[['GEOCODIGO', 'DESBDT', 'geometry']].copy()

# Asignación de cada hotel al barrio al que pertenece
hoteles_por_barrio = gpd.sjoin(
    hoteles_clean,
    barrios_clean,
    how="inner",
    predicate="within"
)

# Recuento de hoteles por barrio (usando GEOCODIGO)
conteo_hoteles = hoteles_por_barrio.groupby('GEOCODIGO').size().reset_index(name='num_hoteles')

print("✅ Recuento de hoteles por barrio final.")
conteo_hoteles.head()

✅ Recuento de hoteles por barrio final.


,GEOCODIGO,num_hoteles
0,079011,20
1,079012,8
2,079013,31
3,079014,11
4,079015,15


In [38]:
# Fusionar el recuento con el GeoDataFrame de barrios
centro_con_hoteles = i_data_centro[['GEOCODIGO', 'DESBDT', 'geometry']].merge(
    conteo_hoteles,
    on='GEOCODIGO',
    how='left'
)

# Rellenar barrios sin hoteles con un 0
centro_con_hoteles['num_hoteles'] = centro_con_hoteles['num_hoteles'].fillna(0).astype(int)

# Renombrar DESBDT a NOMBRE para mayor claridad
centro_con_hoteles = centro_con_hoteles.rename(columns={'DESBDT': 'NOMBRE'})

print("✅ Fusión completada: datos geográficos + conteo de hoteles")
centro_con_hoteles[['NOMBRE', 'num_hoteles']].sort_values(by='num_hoteles', ascending=False).reset_index(drop=True)

✅ Fusión completada: datos geográficos + conteo de hoteles


,NOMBRE,num_hoteles
0,013 Cortes,31
1,016 Sol,29
2,011 Palacio,20
3,015 Universidad,15
4,041 Recoletos,15
5,014 Justicia,11
6,046 Castellana,9
7,012 Embajadores,8
8,035 Jerónimos,7
9,063 Castillejos,7


In [32]:
# Ver columnas del archivo de oficinas de turismo
print("📋 Columnas disponibles en i_data_oficinaturismo:")
print(i_data_oficinaturismo.columns.tolist())

📋 Columnas disponibles en i_data_oficinaturismo:
['CODIGO2', 'NOMBRE', 'DIRECCIÓN', 'C_POSTAL', 'POBLACION', 'TITULAR', 'PAG_WEB', 'UTM_X', 'UTM_Y', 'CMUN', 'geometry']


In [39]:
import folium
from folium import LayerControl

# Crear mapa base centrado en el Centro de Madrid
m = folium.Map(
    location=[40.4168, -3.7038],
    zoom_start=14,
    tiles='CartoDB Positron'
)

# Añadir capa del mapa de calor (choropleth) por número de hoteles por barrio
folium.Choropleth(
    geo_data=centro_con_hoteles,
    data=centro_con_hoteles,
    columns=['GEOCODIGO', 'num_hoteles'],
    key_on='feature.properties.GEOCODIGO',
    fill_color='YlOrRd',
    fill_opacity=0.7,
    line_opacity=0.5,
    legend_name='Número de hoteles por barrio',
    tooltip=folium.GeoJsonTooltip(
        fields=['NOMBRE', 'num_hoteles'],
        aliases=['Barrio:', 'Hoteles:'],
        localize=True
    )
).add_to(m)

# Añadir marcadores para cada oficina de turismo
for idx, row in i_data_oficinaturismo_centro.iterrows():
    popup_text = f"""
    <b>Oficina:</b> {row['NOMBRE']}<br>
    <b>Dirección:</b> {row['DIRECCIÓN']}<br>
    <b>Población:</b> {row['POBLACION']}<br>
    <b>Sitio web:</b> <a href="{row['PAG_WEB']}" target="_blank">Ver página</a>
    """
    folium.Marker(
        location=[row.geometry.y, row.geometry.x],
        popup=folium.Popup(popup_text, max_width=300),
        icon=folium.Icon(color='blue', icon='info-sign', prefix='glyphicon')
    ).add_to(m)

# Añadir control de capas
LayerControl().add_to(m)

# Vista del  mapa
print("🗺️ Mapa de densidad de hoteles en el Centro de Madrid")
m

🗺️ Mapa de densidad de hoteles en el Centro de Madrid


Estadísticos habituales

In [40]:
# Calcular estadísticas clave
total_hoteles = int(centro_con_hoteles['num_hoteles'].sum())
num_barrios = len(centro_con_hoteles)
promedio = centro_con_hoteles['num_hoteles'].mean()
mediana = centro_con_hoteles['num_hoteles'].median()
max_hoteles = int(centro_con_hoteles['num_hoteles'].max())
min_hoteles = int(centro_con_hoteles['num_hoteles'].min())
barrio_top = centro_con_hoteles.loc[centro_con_hoteles['num_hoteles'].idxmax(), 'NOMBRE']

#  Resumen
print("📊 ESTADÍSTICOS HABITUALES: Densidad de hoteles en el Centro de Madrid")
print("=" * 70)
print(f"Total de hoteles en el Centro: {total_hoteles}")
print(f"Número de barrios analizados: {num_barrios}")
print(f"Promedio de hoteles por barrio: {promedio:.2f}")
print(f"Mediana de hoteles por barrio: {mediana:.0f}")
print(f"Mínimo de hoteles en un barrio: {min_hoteles}")
print(f"Máximo de hoteles en un barrio: {max_hoteles}")
print(f"Barrio con más hoteles: {barrio_top}")
print("=" * 70)

# Top 5 barrios con más hoteles
top5 = centro_con_hoteles[['NOMBRE', 'num_hoteles']].sort_values(by='num_hoteles', ascending=False).head(5)
print("🔝 Top 5 barrios con más hoteles:")
print(top5.to_string(index=False))

📊 ESTADÍSTICOS HABITUALES: Densidad de hoteles en el Centro de Madrid
Total de hoteles en el Centro: 214
Número de barrios analizados: 43
Promedio de hoteles por barrio: 4.98
Mediana de hoteles por barrio: 2
Mínimo de hoteles en un barrio: 0
Máximo de hoteles en un barrio: 31
Barrio con más hoteles: 013 Cortes
🔝 Top 5 barrios con más hoteles:
         NOMBRE  num_hoteles
     013 Cortes           31
        016 Sol           29
    011 Palacio           20
015 Universidad           15
  041 Recoletos           15
